In [ ]:
import os
from datasets import load_dataset
from PIL import Image
import pandas as pd
from easyocr import Reader
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import pytesseract
from paddleocr import PaddleOCR
from rapidocr_onnxruntime import RapidOCR
from surya.detection import DetectionPredictor
from surya.recognition import RecognitionPredictor

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None) # Show full content of each column (no truncation)
pd.set_option('display.width', None)

In [ ]:
dataset = load_dataset("ahmedheakl/arocrbench_khatt",split="train")


In [ ]:
results = []
for sample in dataset:
    image = sample["image"]  # Already a PIL Image object
    if image.mode != "RGB":
        image = image.convert("RGB")
    ground_truth = sample["text"]
    results.append({
        "image": image,
        "ground_truth": ground_truth,
        "tesseract": None,
        "easyocr": None,
        "trocr": None,
        "paddleocr": None,
        "rapidocr": None,
        "suryaocr": None
    })
print(f"Initialized results with {len(results)} samples.")

### Tesseract Processing

In [ ]:
for i, sample in enumerate(results):
    image = sample["image"]
    tesseract_text = pytesseract.image_to_string(image,lang='ara')
    results[i]['tesseract'] = tesseract_text
print("Tesseract Processing completeed")

## EasyOCR

In [ ]:
easyocr_reader = Reader(['ar'], gpu=True)
for i, sample in enumerate(results):
    image = sample["image"]
    easyocr_result = easyocr_reader.readtext(image, detail=0)
    easyocr_text = " ".join(easyocr_result)
    results[i]["easyocr"] = easyocr_text
print("EasyOCR processing completed.")

## Docling OCR

In [24]:
df = pd.DataFrame(results)
df.head()


,image,ground_truth,tesseract,easyocr,trocr,paddleocr,rapidocr,suryaocr
0,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2142x141 at 0x7FB164997D90>,وصفاً مجملاً لحالة المغرب خلال سنة 2005، مثيراً ، في سياق ذلك ، التساؤلات الكبرى,"و اث\nحلم خالد\nصف للج نع 1\nٍ حال ل ‎٠‏\n‏2ه م\n06 »م صرمر ا ء ْ\n- ضيبا ماد لأى مع\nالمسشساز ,لي\n0 اح\n","حالا مح٥ً لحالدالمح صف سراء ذ سب مه دلل ا ,لسادلان للورا ها )4~5ه2",None,None,CrD 0,
1,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2159x228 at 0x7FB16497EB30>,وهلال خازن عفيف للحج . بدأت قوافل الحجيج حاج إثر آخر يلبي,"3 ظارن, عض لمع “اأات قاططع لجيج عام رك نم مل\n",عصعا معبل ل _ جا ت قواقل عا 2 إثر ~ خ ما رره للهج المجيج,None,None,2 2,
2,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1508x404 at 0x7FB16497EA70>,النص: مشمش ، دراق ، غيظ ، ناء ، بث ، نسر .,الى : محش ؛ د راك حيظ ؛ دنأد؛ جمبثت اجر\n,"لبث دراد , خيظ ' نا ء مهمم الذم نسلن",None,None,-31,
3,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1236x103 at 0x7FB16497E9E0>,و هي التي جعلته كريما على فقره .,و هى انوا كلت دريما كل موه .\n,لريما ىلى ححلد الد مشظو ه,None,None,它,
4,<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1873x110 at 0x7FB16497E950>,مثل أنقض بغـلس له الضابط لزمتك. سألت راجح هل أبلغ أصحابنا,مثا دمض نغلس هف الطابط لزمتك ساهه راجع هل أبلؤ أصمجاين!ا\n,امجا سالت راجح هل لل الطا ط مث ' نقحى ابل بنا لرمرك نغلل,None,None,CUD 3,


In [ ]:
from jiwer import wer, cer 

# for model in ["tesseract", "easyocr", "trocr"]:
for model in ["tesseract","easyocr"]:
    wer_score = wer(df["ground_truth"].tolist(), df[model].tolist())
    cer_score = cer(df["ground_truth"].tolist(), df[model].tolist())
    print(f"{model} - WER: {wer_score:.2f}, CER: {cer_score:.2f}")

In [ ]:
df.to_csv("khattat_result")